# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3f1474fe50>
├── 'a' --> tensor([[ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048]])
└── 'x' --> <FastTreeValue 0x7f3f1474ff10>
    └── 'c' --> tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                        [ 0.0189,  0.4721, -0.0922,  0.8710],
                        [ 1.2069, -0.5016,  0.2990, -0.0715]])

In [4]:
t.a

tensor([[ 0.4165, -0.6503,  0.5432],
        [-1.0924, -0.6717, -1.0048]])

In [5]:
%timeit t.a

60.9 ns ± 0.0469 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3f1474fe50>
├── 'a' --> tensor([[ 0.1041,  1.6200,  0.8453],
│                   [-0.0112,  0.9996, -0.4959]])
└── 'x' --> <FastTreeValue 0x7f3f1474ff10>
    └── 'c' --> tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                        [ 0.0189,  0.4721, -0.0922,  0.8710],
                        [ 1.2069, -0.5016,  0.2990, -0.0715]])

In [7]:
%timeit t.a = new_value

62.5 ns ± 0.0443 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048]]),
    x: Batch(
           c: tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                      [ 0.0189,  0.4721, -0.0922,  0.8710],
                      [ 1.2069, -0.5016,  0.2990, -0.0715]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4165, -0.6503,  0.5432],
        [-1.0924, -0.6717, -1.0048]])

In [11]:
%timeit b.a

59.4 ns ± 0.0418 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1896,  1.0155,  0.2761],
               [-0.8377,  0.6237, -2.3360]]),
    x: Batch(
           c: tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                      [ 0.0189,  0.4721, -0.0922,  0.8710],
                      [ 1.2069, -0.5016,  0.2990, -0.0715]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.858 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.268 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3e5c9d5b50>
├── 'a' --> tensor([[[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]],
│           
│                   [[ 0.4165, -0.6503,  0.5432],
│                    [-1.0924, -0.6717, -1.0048]]])
└── 'x' --> <FastTreeValue 0x7f3e5c9d59a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 42.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3f146d5370>
├── 'a' --> tensor([[ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048],
│                   [ 0.4165, -0.6503,  0.5432],
│                   [-1.0924, -0.6717, -1.0048]])
└── 'x' --> <FastTreeValue 0x7f3e6358c4f0>
    └── 'c' --> tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                        [ 0.0189,  0.4721, -0.0922,  0.8710],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]],
       
               [[ 0.4165, -0.6503,  0.5432],
                [-1.0924, -0.6717, -1.0048]]]),
    x: Batch(
           c: tensor([[[ 3.0468,  0.2302, -2.4803,  1.9666],
                       [ 0.0189,  0.4721, -0.0922,  0.8710],
                       [ 1.2069, -0.5016,  0.2990, -0.0715]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048],
               [ 0.4165, -0.6503,  0.5432],
               [-1.0924, -0.6717, -1.0048]]),
    x: Batch(
           c: tensor([[ 3.0468,  0.2302, -2.4803,  1.9666],
                      [ 0.0189,  0.4721, -0.0922,  0.8710],
                      [ 1.2069, -0.5016,  0.2990, -0.0715],
                      [ 3.0468,  0.2302, -2.4803,  1.9666],
                      [ 0.0189,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
